In [2]:
#!pip install -qU ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
cd titanic

/kaggle/working/titanic


In [25]:
import numpy as np
import pandas as pd

path = "/kaggle/input/competitions/titanic/train.csv"
df = pd.read_csv(path)

df["Pclass"] = df["Pclass"] - 1
df["Age"] = df["Age"].fillna(value=df["Age"].mean())

df = pd.get_dummies(df, columns=["Embarked", "Cabin"], dummy_na=True, dtype=np.int64)
df["Sex"] = df["Sex"].map({"male": 0, "female": 1})

df.drop(["Name", "Ticket", "PassengerId"], axis=1, inplace=True)

In [28]:
target = df["Survived"]
df.drop(["Survived"], axis=1, inplace=True)
inputs = df.to_numpy()

In [32]:
from sklearn.model_selection import train_test_split

SEED = 63166

train_inputs, eval_inputs, train_target, eval_target = train_test_split(inputs, 
                                                                       target,
                                                                       test_size=0.2,
                                                                       random_state=SEED)

In [35]:
np.bincount(train_target)

array([449, 263])

In [36]:
np.bincount(eval_target)

array([100,  79])